# Creditworthiness analysis


Based on the data from the bank's credit department, I investigated the influence of marital status and the number of children on the timely repayment of loans. Information about the data was obtained, and missing values were identified and processed. Data types were adjusted to match the stored data. Duplicates were removed. The data was categorized, and one dataframe was decomposed into three.

## Let's take a look at the data

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Data Preprocessing

### Dealing with missing values

In [4]:
# percent of missing values by column
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# filling missing values in "total_income" with a median based on income type
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Dealing with anomalies

In [6]:
# replacing negative values with absolute value
data['days_employed'] = data['days_employed'].abs()

In [7]:
# calculating the median value of the "days_employed" for each employment type
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

There are abnormally large values for two types: unemployed and retirees. Fixing such values is difficult, so leaving them as they are.

In [8]:
# unique values in "children"
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
# removing rows with abnormal values
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Processing the remaining missing values

Let's fill in the missing values in the "days_employed" column with the median values for each income_type.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
# checking that there are no missing values left
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Data Type Conversion

Let's convert the data type of the "total_income" column from float to integer using the `astype()` method.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Duplicates processing

Let's handle implicit duplicates in the 'education' column. In this column, there are identical values but written differently, using both uppercase and lowercase letters. We will convert them to lowercase. Next, we will check the remaining columns for duplicates.

In [14]:
data['education'] = data['education'].str.lower()

Let's print the number of duplicate rows in the data and remove them if any exist.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Data categorization

Let's create the "total_income_category" column in the "data" DataFrame based on the specified income ranges. We will use a custom function called "categorize_income()" and apply it using the "apply()" method. The function will categorize the incomes according to the given ranges and assign the corresponding category ('E', 'D', 'C', 'B', or 'A').

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Let's print the unique values of the "purpose" column to display the list of unique credit purposes in the dataset.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

To create the new column purpose_category based on the data from the purpose column, we can define a custom function called categorize_purpose() and use the apply() method to apply this function to each row in the purpose column.

The categories are:

- `'операции с автомобилем'` ("car"),
- `'операции с недвижимостью'` ("real estate"),
- `'проведение свадьбы'` ("wedding"),
- `'получение образования'` ("education").

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Analysis and conclusions

#### Is there a correlation between the number of children and loan repayment on time?

In [22]:
# creating a function for calculating the percentage ratio of a sum to a quantity
def ratio(value):
    return str(round((value.sum() / value.count()) * 100, 2)) + '%'

# creating a pivot table
pivoted = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum', ratio])

# the result in descending order of the percentage of overdue loans
pivoted.sort_values(by=('ratio', 'debt'), ascending = False)

,count,sum,ratio
,debt,debt,debt
children,,,
4,41,4,9.76%
2,2052,194,9.45%
1,4808,444,9.23%
3,330,27,8.18%
0,14091,1063,7.54%
5,9,0,0.0%


There is a correlation between the number of children and loan repayment: in general, clients with more children tend to have a higher number of overdue payments. However, clients with 5 children have no overdue payments, but there are only 9 such families in the sample, which makes the data for them less representative. Also, families with three children have a better repayment rate compared to families with one child. For a more accurate analysis, it would be helpful to have a larger sample of clients with three children. Clients with no children have the lowest percentage of overdue payments, excluding families with 5 children.

#### Is there a correlation between marital status and timely loan repayment?

In [23]:
# creating a pivot table
pivoted = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', ratio])

# the result in descending order of the percentage of overdue loans
pivoted.sort_values(by=('ratio', 'debt'), ascending = False)

,count,sum,ratio
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,9.76%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%


In [24]:
# calculating median age of widows/widowers
int(data.loc[data['family_status']=='вдовец / вдова', 'dob_years'].mean())

56

There is a correlation between marital status and timely loan repayment. The highest percentage of clients with overdue loans is among unmarried individuals. Clients who are or have been married have better repayment rates. The lowest percentage of overdue loans is among widows and widowers, but this category is the least represented. Since the median age among widowed individuals is 56 years, it can be assumed that they mostly have stable financial situations, which contributes to on-time loan repayment.


#### Is there a correlation between income level and loan repayment on time?

In [25]:
# categories:
# 0–30000 — 'E';
# 30001–50000 — 'D';
# 50001–200000 — 'C';
# 200001–1000000 — 'B';
# 1000001 and higher — 'A'.

# creating a pivot table
pivoted = data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum', ratio])

# the result in descending order of the percentage of overdue loans
pivoted.sort_values(by=('ratio', 'debt'), ascending = False)

,count,sum,ratio
,debt,debt,debt
total_income_category,,,
E,22,2,9.09%
C,15921,1353,8.5%
A,25,2,8.0%
B,5014,354,7.06%
D,349,21,6.02%


The probability of loan default is highest among clients with income less than 30,000. For clients whose income exceeds this amount, there are no clear patterns or correlations with loan repayment.

#### How different credit purposes affect their timely repayment?

In [26]:
# creating a pivot table
pivoted = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum', ratio])

# the result in descending order of the percentage of overdue loans
pivoted.sort_values(by=('ratio', 'debt'), ascending = False)

,count,sum,ratio
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.35%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%
операции с недвижимостью,10751,780,7.26%


Clients whose purpose is "operations with a car" or "education" are the most likely to have delays in loan repayment. On the other hand, loans for "real estate transactions" and "wedding expenses" are more often repaid on time.

#### Possible reasons for the appearance of missing values in the original data.

For example, a user may not have provided certain information about themselves, or an automated data collection system may have experienced a failure. It is also possible that missing values were intentionally left blank, possibly expecting automated filling with special characters. Various reasons can lead to missing values in the data, and understanding the nature of these missing values is crucial for accurate data analysis and interpretation.

### Summary

**Research results:**

1. The hypothesis regarding the correlation between the number of children and on-time loan repayment is generally confirmed: the more children a borrower has, the more likely they are to delay loan repayment. However, the difference between clients without children and clients with 1 or 2 children is not significant: 7.54% vs. 9.23% and 9.45%, respectively. Families with many children are underrepresented in the sample (380), compared to families with 1-2 children and childless families (20951 together). More data is needed to validate the hypothesis for families with many children.

2. Marital status indeed affects loan repayment behavior, and the hypothesis was correct. Clients who have never been married are more likely to default on loans. However, the difference between the group of married clients and the groups in civil partnerships or not in any partnership is not significant, about 2%.

3. Comparing the two most numerous income groups in the sample 'C' (income 50001–200000) and 'B' (income 200001–1000000), it can be concluded that clients with higher incomes are more likely to repay loans on time. Therefore, the hypothesis is valid for these groups. However, groups A (highest income), E (lowest income), and D (30001-50000) are too small to draw precise conclusions.

4. The relationship between the purpose of the loan and the likelihood of on-time repayment is confirmed. Loans related to real estate (7.26%) and wedding expenses (7.91%) are most likely to be repaid on time. On the other hand, loans for cars (9.35%) and education (9.25%) have a lower on-time repayment rate. Although there is a clear influence of the loan purpose, the difference in results is small, within approximately 2%.

**Recommendations:**

- Since clear trends are observed in the results of hypotheses #2 and #4, they should be taken into account when developing a scoring system.
- More data is needed for families with many children and for income groups A, E, and D to revalidate hypotheses #1 and #3.